In [144]:
#import the libraries you'll need

import pandas as pd
import numpy as np
import pickle
from sklearn import impute

## Let's walk through the other methods of handling missing data:


 
 - Fill with zeros/ones
 - Fill with True/False
 - Fill with mode/median
 - Fill with interpolated values
 - Fill forward/backward
 - Fill randomly
 

It's worth mentioning here, that not all of these options are reccomended or equally valid. Choosing your imputation methodology will affect the underlying distribution of the data, and can have profound impacts downstream. There are a huge number of things to consider when imputing data. This lab is about the tchnical HOW of data imputation. The when/why/where questions are much more complex and generally require some level of subject matter expertise on the underlying data, or at least some level of statistical understanding.

High level, there are a few things you wan't to think about when dealing with missing data:

 - Is the data missing at complete random?
 - Is there an underlying cause to the "missingness". In other words, does the underlying value drive its likeliness to be missing?
 - Is the missing data related to the other data points for that observation?
 
One of UBER's data scientists has done a nice write up of some of these considerations [here](https://towardsdatascience.com/how-to-handle-missing-data-8646b18db0d4)

There is on oft quoted snippet from two Harvard statistics professors that sums up the dangers of imputation nicely:

“The idea of imputation is both seductive and dangerous. It is seductive because it can lull the user into the pleasurable state of believing that the data are complete after all, and it is dangerous because it lumps together situations where the problem is sufficiently minor that it can legitimately handled in this way and situations where standard estimators applied to real and imputed data have substantial bias.”

        ----Donald Rubin and Arthur Dempster (Incomplete Data in Sample Surveys: Theory and bibliographies)

In [141]:
#import the .pickle file we made in part 4

data = pd.read_pickle('credit_data_with_nans.pickle')
data.head()

ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,20000.0,2.0,2.0,1.0,24.0,NaN,2.0,NaN,-1.0,-2.0,...,NaN,NaN,NaN,NaN,689.0,NaN,NaN,0.0,0.0,NaN
2,120000.0,2.0,2.0,NaN,26.0,-1.0,NaN,0.0,0.0,NaN,...,3272.0,3455.0,3261.0,0.0,NaN,1000.0,NaN,0.0,2000.0,1.0
3,90000.0,2.0,NaN,2.0,34.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,1518.0,1500.0,1000.0,1000.0,NaN,5000.0,0.0
4,50000.0,2.0,2.0,1.0,37.0,NaN,0.0,0.0,0.0,0.0,...,28314.0,28959.0,29547.0,NaN,2019.0,1200.0,1100.0,1069.0,1000.0,0.0
5,50000.0,1.0,2.0,1.0,NaN,-1.0,0.0,-1.0,0.0,0.0,...,20940.0,19146.0,19131.0,NaN,36681.0,10000.0,9000.0,689.0,NaN,NaN


In [3]:
#fill the NaNs with zeros in a new df.

zeros = data.fillna(0)
zeros.head()

ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,20000.0,2.0,2.0,1.0,24.0,0.0,2.0,0.0,-1.0,-2.0,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,0.0
2,120000.0,2.0,2.0,0.0,26.0,-1.0,0.0,0.0,0.0,0.0,...,3272.0,3455.0,3261.0,0.0,0.0,1000.0,0.0,0.0,2000.0,1.0
3,90000.0,2.0,0.0,2.0,34.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1518.0,1500.0,1000.0,1000.0,0.0,5000.0,0.0
4,50000.0,2.0,2.0,1.0,37.0,0.0,0.0,0.0,0.0,0.0,...,28314.0,28959.0,29547.0,0.0,2019.0,1200.0,1100.0,1069.0,1000.0,0.0
5,50000.0,1.0,2.0,1.0,0.0,-1.0,0.0,-1.0,0.0,0.0,...,20940.0,19146.0,19131.0,0.0,36681.0,10000.0,9000.0,689.0,0.0,0.0


In [4]:
#fill the NaNs with either True or False in a new df

false = data.fillna(False)
false.head()

ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,20000,2,2,1,24,False,2,False,-1,-2,...,False,False,False,False,689,False,False,0,0,False
2,120000,2,2,False,26,-1,False,0,0,False,...,3272,3455,3261,0,False,1000,False,0,2000,1
3,90000,2,False,2,34,0,0,0,0,0,...,False,False,False,1518,1500,1000,1000,False,5000,0
4,50000,2,2,1,37,False,0,0,0,0,...,28314,28959,29547,False,2019,1200,1100,1069,1000,0
5,50000,1,2,1,False,-1,0,-1,0,0,...,20940,19146,19131,False,36681,10000,9000,689,False,False


In [4]:
#fill the NaNs with either the mode or median of each respective column

median = data.fillna(data.median())
median.head()

ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,20000.0,2.0,2.0,1.0,24.0,0.0,2.0,0.0,-1.0,-2.0,...,19023.0,18158.0,17040.0,2100.0,689.0,1800.0,1500.0,0.0,0.0,0.0
2,120000.0,2.0,2.0,2.0,26.0,-1.0,0.0,0.0,0.0,0.0,...,3272.0,3455.0,3261.0,0.0,2014.0,1000.0,1500.0,0.0,2000.0,1.0
3,90000.0,2.0,2.0,2.0,34.0,0.0,0.0,0.0,0.0,0.0,...,19023.0,18158.0,17040.0,1518.0,1500.0,1000.0,1000.0,1500.0,5000.0,0.0
4,50000.0,2.0,2.0,1.0,37.0,0.0,0.0,0.0,0.0,0.0,...,28314.0,28959.0,29547.0,2100.0,2019.0,1200.0,1100.0,1069.0,1000.0,0.0
5,50000.0,1.0,2.0,1.0,34.0,-1.0,0.0,-1.0,0.0,0.0,...,20940.0,19146.0,19131.0,2100.0,36681.0,10000.0,9000.0,689.0,1500.0,0.0


In [5]:
#fill the NaNs with interpolated values. Double check the NaN counts after doing this and see if anything seems off.

interpolated = data.interpolate()
interpolated.head()

ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,20000.0,2.0,2.0,1.0,24.000000,NaN,2.0,NaN,-1.0,-2.0,...,NaN,NaN,NaN,NaN,689.0,NaN,NaN,0.0,0.0,NaN
2,120000.0,2.0,2.0,1.5,26.000000,-1.0,1.0,0.0,0.0,-1.0,...,3272.0,3455.0,3261.0,0.0,1094.5,1000.0,NaN,0.0,2000.0,1.0
3,90000.0,2.0,2.0,2.0,34.000000,0.0,0.0,0.0,0.0,0.0,...,15793.0,16207.0,16404.0,1518.0,1500.0,1000.0,1000.0,534.5,5000.0,0.0
4,50000.0,2.0,2.0,1.0,37.000000,-0.5,0.0,0.0,0.0,0.0,...,28314.0,28959.0,29547.0,14888.5,2019.0,1200.0,1100.0,1069.0,1000.0,0.0
5,50000.0,1.0,2.0,1.0,34.333333,-1.0,0.0,-1.0,0.0,0.0,...,20940.0,19146.0,19131.0,28259.0,36681.0,10000.0,9000.0,689.0,900.0,0.0


In [6]:
#Show the NaN counts for the interpolated df. Why are there still a few straggler NaNs?

interpolated.isna().sum()

ID
LIMIT_BAL                     0
SEX                           0
EDUCATION                     0
MARRIAGE                      0
AGE                           0
PAY_0                         1
PAY_2                         0
PAY_3                         1
PAY_4                         0
PAY_5                         0
PAY_6                         0
BILL_AMT1                     0
BILL_AMT2                     2
BILL_AMT3                     0
BILL_AMT4                     1
BILL_AMT5                     1
BILL_AMT6                     1
PAY_AMT1                      1
PAY_AMT2                      0
PAY_AMT3                      1
PAY_AMT4                      2
PAY_AMT5                      0
PAY_AMT6                      0
default payment next month    1
dtype: int64

In [7]:
##fill the NaNs with the preceding values

preceding = data.fillna(method='bfill')
preceding.head()

ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,20000.0,2.0,2.0,1.0,24.0,-1.0,2.0,0.0,-1.0,-2.0,...,3272.0,3455.0,3261.0,0.0,689.0,1000.0,1000.0,0.0,0.0,1.0
2,120000.0,2.0,2.0,2.0,26.0,-1.0,0.0,0.0,0.0,0.0,...,3272.0,3455.0,3261.0,0.0,1500.0,1000.0,1000.0,0.0,2000.0,1.0
3,90000.0,2.0,2.0,2.0,34.0,0.0,0.0,0.0,0.0,0.0,...,28314.0,28959.0,29547.0,1518.0,1500.0,1000.0,1000.0,1069.0,5000.0,0.0
4,50000.0,2.0,2.0,1.0,37.0,-1.0,0.0,0.0,0.0,0.0,...,28314.0,28959.0,29547.0,55000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0.0
5,50000.0,1.0,2.0,1.0,29.0,-1.0,0.0,-1.0,0.0,0.0,...,20940.0,19146.0,19131.0,55000.0,36681.0,10000.0,9000.0,689.0,800.0,0.0


In [8]:
##fill the NaNs with the following values

following = data.fillna(method='ffill')
following.head()

ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,20000.0,2.0,2.0,1.0,24.0,NaN,2.0,NaN,-1.0,-2.0,...,NaN,NaN,NaN,NaN,689.0,NaN,NaN,0.0,0.0,NaN
2,120000.0,2.0,2.0,1.0,26.0,-1.0,2.0,0.0,0.0,-2.0,...,3272.0,3455.0,3261.0,0.0,689.0,1000.0,NaN,0.0,2000.0,1.0
3,90000.0,2.0,2.0,2.0,34.0,0.0,0.0,0.0,0.0,0.0,...,3272.0,3455.0,3261.0,1518.0,1500.0,1000.0,1000.0,0.0,5000.0,0.0
4,50000.0,2.0,2.0,1.0,37.0,0.0,0.0,0.0,0.0,0.0,...,28314.0,28959.0,29547.0,1518.0,2019.0,1200.0,1100.0,1069.0,1000.0,0.0
5,50000.0,1.0,2.0,1.0,37.0,-1.0,0.0,-1.0,0.0,0.0,...,20940.0,19146.0,19131.0,1518.0,36681.0,10000.0,9000.0,689.0,1000.0,0.0


In [19]:
##fill the NaNs with the randomly sampled values

random = data.fillna(data.sample(1))
random.head()

ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,20000.0,2.0,2.0,1.0,24.0,NaN,2.0,NaN,-1.0,-2.0,...,NaN,NaN,NaN,NaN,689.0,NaN,NaN,0.0,0.0,NaN
2,120000.0,2.0,2.0,NaN,26.0,-1.0,NaN,0.0,0.0,NaN,...,3272.0,3455.0,3261.0,0.0,NaN,1000.0,NaN,0.0,2000.0,1.0
3,90000.0,2.0,NaN,2.0,34.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,1518.0,1500.0,1000.0,1000.0,NaN,5000.0,0.0
4,50000.0,2.0,2.0,1.0,37.0,NaN,0.0,0.0,0.0,0.0,...,28314.0,28959.0,29547.0,NaN,2019.0,1200.0,1100.0,1069.0,1000.0,0.0
5,50000.0,1.0,2.0,1.0,NaN,-1.0,0.0,-1.0,0.0,0.0,...,20940.0,19146.0,19131.0,NaN,36681.0,10000.0,9000.0,689.0,NaN,NaN


As we mentioned briefly in the last lab, you've probably noticed by now that none of the above methods make sense for categorical variables. So, let's get more granular and apply some column level handling of NaNs.

In [23]:
#randomly fill the NaNs in the Marriage column with values

values = data.MARRIAGE.unique()[np.logical_not(np.isnan(data.MARRIAGE.unique()))]

data.MARRIAGE = data.MARRIAGE.fillna(np.random.choice(values))

In [114]:
data.MARRIAGE.fillna(data.MARRIAGE.sample(len(data)), inplace=True)

In [133]:
data.MARRIAGE.value_counts()

2.0    11983
1.0    10265
3.0      244
0.0       43
Name: MARRIAGE, dtype: int64

In [123]:
data.MARRIAGE.value_counts()

2.0    11983
1.0    10265
3.0      244
0.0       43
Name: MARRIAGE, dtype: int64

In [122]:
data.MARRIAGE.isna().sum()

7465

It's worth noting that the fillna() method defaults to filling along the index axis. If you'd like to fill along the column axis, just use the axis= kwarg. Reading the docs for fillna() is worthwhile here.

Now that we've gone through the various methi